<a href="https://colab.research.google.com/github/Komal77rao/Data-Eng-Modules/blob/main/adventureworks-1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# AdventureWorks Relational Postgres Lab

### Introduction

In this lesson, we will work with the adventureworks database in postgres.  Let's get started.

### Loading our data


We can begin by making sure our postgres application is running (if on a Mac, look for the elephant at the top right of the screen).  Once running we'll walk through the instructions in the [Adventureworks for postgres](https://github.com/lorint/AdventureWorks-for-Postgres) repo below.

First move into the `install-script` directory.  Then run the following:

```bash
psql -c "CREATE DATABASE \"adventureworks\";"
psql -d adventureworks < install.sql
```

Next connect to the adventureworks database.

```sql
\c "Adventureworks"
```

One confusing thing is that if we simply type `\dt` to display our tables, we won't find any listed.  This is because all of our tables are under different schemas.  We can see this if, while connected to the adventureworks database, we run the following:

```sql
\dt (humanresources|person|production|purchasing|sales).*
```

So, as we can see, there are indeed a lot of tables in our database.  And we can query any of those tables with the schema prefix like so.

```sql
select * from person.address limit 1;
```

The database is pretty complicated, and you can see all of the relations from the [erd link](https://i0.wp.com/improveandrepeat.com/wp-content/uploads/2018/12/AdvWorksOLTPSchemaVisio.png?ssl=1).  But we will stick to the tables in the `sales` schema.  Let's get started.



### Beginning our queries

We can connect to our database using the postgres library.

In [ ]:
import pandas as pd
import psycopg2

conn = psycopg2.connect(database="Adventureworks", user="postgres")

And from there, we can use pandas to read from our database.  We'll can begin with our customers table.

In [ ]:
pd.read_sql("select * from sales.Customer limit 5", conn)

/usr/local/lib/python3.9/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,customerid,personid,storeid,territoryid,rowguid,modifieddate
0,1,None,934,1,3f5ae95e-b87d-4aed-95b4-c3797afcb74f,2014-09-12 11:15:07.263
1,2,None,1028,1,e552f657-a9af-4a7d-a645-c429d6e02491,2014-09-12 11:15:07.263
2,3,None,642,4,130774b1-db21-4ef3-98c8-c104bcd6ed6d,2014-09-12 11:15:07.263
3,4,None,932,4,ff862851-1daa-4044-be7c-3e85583c054d,2014-09-12 11:15:07.263
4,5,None,1026,4,83905bdc-6f5e-4f71-b162-c98da069f38a,2014-09-12 11:15:07.263


We can hide warnings by running the following code.

In [ ]:
import warnings
warnings.filterwarnings('ignore')

And then let's view some of the data in our `SalesOrderHeader` table.

> Notice below we have a `.T` at the very end.  This is to `transpose` our dataframe -- which sometimes makes it easier to see all of the columns.  The `0` and `1` are to describe each of the rows of data.

In [ ]:
pd.read_sql("select * from sales.SalesOrderHeader limit 2", conn).T

So we can see that our `SalesOrderHeader` table has a good amount of foreign keys including `customerId`.

The SalesOrderHeader table is one key table.  Another is the `salesOrderDetail` table.  Make sure to take a look at the [erd link](https://i0.wp.com/improveandrepeat.com/wp-content/uploads/2018/12/AdvWorksOLTPSchemaVisio.png?ssl=1) to get a sense of how the tables and others in the sales schema relate to one another.

### Writing some queries

Noe let's get a sense of our data.  First, find the earliest date of our orders in the `SalesOrderHeader` table.

In [ ]:
import pandas as pd
query = "select min(orderdate) from sales.SalesOrderHeader"
pd.read_sql(query, conn)

# 	min
# 0	2011-05-31

And next find the latest date of the orders.

In [ ]:
query =  "select max(orderdate) from sales.SalesOrderHeader"
pd.read_sql(query, conn)


# max
# 0	2014-06-30

,max
0,2014-06-30


We can see that our most recent orders were placed at the end June of 2014.  

Next find the the top five total amounts spent by customers since July 1 2013.  Return both the customerIds and the total amounts spent.

In [ ]:
query = """
select customerids, sum(taxamt)
from sales.SalesOrderHeader
where orderdate >= 2013-07-01
group by customerids


 """
pd.read_sql(query, conn)

# 	customerid	total_amount
# 0	29641	419331.7837
# 1	29629	389437.6577
# 2	29923	386059.4979
# 3	29736	363872.0389
# 4	29701	355051.0477

,customerid,total_amount
0,29641,419331.7837
1,29629,389437.6577
2,29923,386059.4979
3,29736,363872.0389
4,29701,355051.0477


Then find the names of the top five products that brought in the most amount of revenue, and the amount of revenue for each since `July 1st 2013` -- we can use the `salesOrderDetail.modifiedDate` column as a proxy for the dates of each sale.

In [ ]:
query = """
"""
pd.read_sql(query, conn)

# name	total_price
# 0	Mountain-200 Black, 38	1.259927e+06
# 1	Mountain-200 Black, 42	1.222931e+06
# 2	Mountain-200 Silver, 38	1.156979e+06
# 3	Mountain-200 Black, 46	1.141849e+06
# 4	Mountain-200 Silver, 46	1.128350e+06

,name,total_price
0,"Mountain-200 Black, 38",1.259927e+06
1,"Mountain-200 Black, 42",1.222931e+06
2,"Mountain-200 Silver, 38",1.156979e+06
3,"Mountain-200 Black, 46",1.141849e+06
4,"Mountain-200 Silver, 46",1.128350e+06


So now that we've done some analysis on the products, let's take a look at the regions.  Find the territoryids related to the territories that have had the most sales in the past year.

In [ ]:
query = """
"""
pd.read_sql(query, conn)

# name	total_amount
# 0	Southwest	9.994438e+06
# 1	Northwest	7.492804e+06
# 2	Canada	6.367257e+06
# 3	Australia	6.154403e+06
# 4	United Kingdom	5.047151e+06
# 5	France	4.370710e+06
# 6	Germany	3.630621e+06
# 7	Central	2.791293e+06
# 8	Southeast	2.351479e+06
# 9	Northeast	2.168895e+06

,name,total_amount
0,Southwest,9.994438e+06
1,Northwest,7.492804e+06
2,Canada,6.367257e+06
3,Australia,6.154403e+06
4,United Kingdom,5.047151e+06
5,France,4.370710e+06
6,Germany,3.630621e+06
7,Central,2.791293e+06
8,Southeast,2.351479e+06
9,Northeast,2.168895e+06


### Looking at Salespeople

Next, let's try to find the top performing salespeople.  We can begin by looking at our salesperson table.

In [ ]:
pd.read_sql("""select * from sales.SalesPerson limit 2""", conn)

,businessentityid,territoryid,salesquota,bonus,commissionpct,salesytd,saleslastyear,rowguid,modifieddate,territoryid,name,countryregioncode,group,salesytd,saleslastyear,costytd,costlastyear,rowguid,modifieddate
0,275,2,300000.0,4100.0,0.012,3.763178e+06,1.750406e+06,1e0a7274-3064-4f58-88ee-4c6586c87169,2011-05-24,2,Northeast,US,North America,2.402177e+06,3.607149e+06,0.0,0.0,00fb7309-96cc-49e2-8363-0a1ba72486f2,2008-04-30
1,276,4,250000.0,2000.0,0.015,4.251369e+06,1.439156e+06,4dd9eee4-8e81-4f8c-af97-683394c1f7c0,2011-05-24,4,Southwest,US,North America,1.051085e+07,5.366576e+06,0.0,0.0,dc3e9ea0-7950-4431-9428-99dbcbc33865,2008-04-30


We can see that we have a `salesytd` column as well as a `modifieddate` column.  One item that may be difficult to identify is the sales person id.  This it turns out is the `businessentityid`.  

Now let's find the salesperson id salesytd of the salesperson who had the most sales in the `Southwest` territory, our best performing territory.

In [ ]:
query = """
select s.businessentityid, s.salesytd
from sales.SalesPerson s
join sales.SalesTerritory t
on s.territoryID = t.territoryID
where t.territoryID = 4
order by salesyt desc
limit 1
"""
print(pd.read_sql(query, conn))


#  businessentityid	salesytd
# 0	276	4.251369e+06

,businessentityid,salesytd
0,276,4.251369e+06


Then let's the top three items that the salesperson with id 276 sold.  

> Some of the foreign keys may be tricky to figure out but use your best guess.

In [ ]:

query = """
select s.unitprice, p.name
from sales.SalesOrderDetail s
join production.Product p on p.productID = s.productID
join sales.SalesOrderHeader h on s.salesorderid = h.salesorderid
where h.salesPersonid = 276
order by h.taxamt desc
limit 3
"""
print(pd.read_sql(query, conn))

# 	amount	name
# 0	141	Long-Sleeve Logo Jersey, L
# 1	135	AWC Logo Cap
# 2	124	Sport-100 Helmet, Blue

,amount,name
0,141,"Long-Sleeve Logo Jersey, L"
1,135,AWC Logo Cap
2,124,"Sport-100 Helmet, Blue"


### Summary

In this lesson, we practiced working with our relational queries.  The adventureworks database is excellent for practicing postgres skills so feel free to continue practicing and exploring the various schemas.